In [ ]:
# 2D Integration using a simplified Finite Element Method

## Theory 

The aim of this task is to integrate a function within a given triangulated area. 

### Refresher: 1 Dimensional Integration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as ptc
from matplotlib import cm

In [ ]:
#i have no idea how subplots work
f = lambda x: x*x

x = np.linspace(-10,10)
for i in range(-10,10,1): 
    rect = ptc.Rectangle((i,0), 1, (f(i)+f(i+1))/2, linewidth=1, edgecolor='gray', facecolor='none')
    plt.gca().add_patch(rect)
plt.plot(x,f(x),c='r',label='f(x)=x^2')
plt.ylim(0,110)
plt.xlim(-10,10)
plt.legend()
plt.show()

### How will 2 dimensional integration work then? 

In [ ]:
with open(r"C:\Users\backl\OneDrive\Desktop\dolphinCoords.txt", 'r') as f: 
    x,y,z = f.readlines()
    x,y,z = x.split(','),y.split(','),z.split(',')
    x,y,z = [float(i) for i in x],[float(i) for i in y],[float(i) for i in z]

fig, ax = plt.subplots(subplot_kw={"projection": "3d"},figsize=(7,7))

surf = ax.plot_trisurf(x, y, z, cmap=cm.coolwarm, linewidth=0, antialiased=False)

fig.colorbar(surf, shrink=0.5, aspect=5)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

#### How did we arrive at that expression? 

We want to compute the area $I_\Omega = \int f(x,y)dxdy$ where $\Omega \in \mathbb{R}^2$

For this purpose the area we want to integrate is divided into triangles, $T_i$. 

The unit triangle can be approximated as follows: 

$$
\int_{T_{u}} f(x,y)dxdy \approx \frac{1}{2} \left( \frac{f(0,0)+f(1,0)+f(0,1)}{3}\right)
$$



But how do we transform a triangle $T_i$ to the unit triangle $T_u$? 

$$
\int_{T_i} f(x,y)dxdy = |J|\int_{T_u}f(x(\xi,\eta),y(\xi,\eta))dxdy
$$

$$
x(\xi,\eta) = x_1 + (x_2-x_1)\xi + (x_3-x_1)\eta \newline
y(\xi,\eta) = y_1 + (y_2-y_1)\xi + (y_3-y_1)\eta
$$


Thus

$$
\int_{T_{i}} f(x,y)dxdy \approx |J|\frac{1}{2} \left( \frac{f(0,0)+f(1,0)+f(0,1)}{3}\right)=|J|\frac{1}{2} \left( \frac{f(x_1,y_1)+f(x_2,y_2)+f(x_3,y_3)}{3}\right)
$$

and from this we can sum all the triangles and get the volume of the integrated function within the triangulated area. 

In [ ]:
#read the coordinates from file -> list of coordinate pairs on the form (point,point)
def ReadCoordFile(path: str) -> list: 
    #read file 
    with open(path,'r') as f: 
        x,y = f.readlines()
        x,y = x.split(),y.split()

    coordinates = []
    #add coordinate pairs
    for ind,_ in enumerate(x): 
        coordinates.append((float(x[ind]),float(y[ind])))
    #-> list of coordinate pairs
    return coordinates

#read node triplets from file -> list of triangle objects 
def ReadNodeFile(path: str, coordinates: list, func) -> list:
    with open(path, 'r') as f: 
        
        p1,p2,p3 = f.readlines()
        p1,p2,p3 = p1.split(),p2.split(),p3.split()

    triangles = []
    #the class triangle takes three coordinate pairs and the function
    for ind,_ in enumerate(p1): 
        triangles.append(Triangle(
            coordinates[int(float(p1[ind])) - 1],
            coordinates[int(float(p2[ind])) - 1],
            coordinates[int(float(p3[ind])) - 1],
            func))
    #-> list of triangle objects
    return triangles

In [ ]:
#the triangle class
class Triangle: 
    #initialization
    def __init__(self, corner1: point, corner2: point, corner3: point, func): 
        self.corner1 = corner1
        self.corner2 = corner2
        self.corner3 = corner3

        self.func = func

        if self.TooSmallAngle(): 
            raise Exception('Angle in triangle too small')
    #jacobian determinant 
    def JacobianDeterminant(self) -> float: 
        return abs(la.det(np.array([[self.corner2[0] - self.corner1[0], self.corner3[0] - self.corner1[0]],
                                    [self.corner2[1] - self.corner1[1], self.corner3[1] - self.corner1[1]]])))

    def TooSmallAngle(self) -> bool: 
        return False
    #implementation of the formula provided
    def VolumeOfPrism(self) -> float: 
        return self.JacobianDeterminant() * (1/6) * (self.func(*self.corner1) + self.func(*self.corner2) + self.func(*self.corner3)) 
    #for summation in the mesh class
    def __add__(self, other) -> float: 
        return self.VolumeOfPrism() + other

    def __radd__(self, other) -> float: 
        return self.VolumeOfPrism() + other



In [ ]:
#mesh class 

class Mesh: 
    #initialized with the paths to coords and nodes who are used to immediately read the files
    def __init__(self, nodePath: str, coordPath: str, func) -> None:
        self.func = func
        self.points = ReadCoordFile(coordPath)
        self.triangles = ReadNodeFile(nodePath,self.points,self.func)
        
    #since Triangle instances can both add and radd it is only a matter of summing the instances get total surface 
    def TotalTriangleArea(self) -> float:
        return sum(self.triangles)

In [ ]:
#main 
def main() -> None:  
    #function that is integrated
    f = lambda x, y : 1
    #mesh instance
    mesh = Mesh('meshes/nodes_unitcircle_10000.txt', 'meshes/coordinates_unitcircle_10000.txt', f)
    #print surface of triangles
    print(mesh.TotalTriangleArea())